In [1]:
import nltk,os
nltk.download('gutenberg')

from nltk.corpus import gutenberg
import pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')

os.makedirs('data',exist_ok=True)

with open('data/hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to C:\Users\Rishad
[nltk_data]     R\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
with open('data/hamlet.txt','r') as file:
    text=file.read().lower()
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(total_words)

4818


In [ ]:
tokenizer.word_index

In [5]:
inputsequences=[]

for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequences=token_list[:i+1]
        inputsequences.append(n_gram_sequences)

In [ ]:
inputsequences

In [7]:
max_len=max([len(x) for x in inputsequences])
max_len

14

In [8]:
input_sequences=np.array(pad_sequences(inputsequences,maxlen=max_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [9]:
x,y=input_sequences[:,:-1],input_sequences[:,-1]
import tensorflow as tf

y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [10]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [12]:
x_train

array([[   0,    0,    0, ...,   46,  723,    2],
       [   0,    0,    0, ...,   54, 3663,  321],
       [   0,    0,    0, ...,    0,    0, 1531],
       ...,
       [   0,    0,    0, ...,    0,    0,   57],
       [   0,    0,    0, ...,  397,  133,  309],
       [   0,    0,    0, ...,   19,   11,    5]])

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(total_words,activation='softmax'))

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [22]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_5 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_2 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               100400    
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219

In [23]:
history=model.fit(x_train,y_train,
                  epochs=50,
                  validation_data=(x_test,y_test),
                  verbose=1,
                  callbacks=[earlystopping])

Epoch 1/50
644/644 [==============================] - 26s 33ms/step - loss: 6.8250 - accuracy: 0.0319 - val_loss: 6.6583 - val_accuracy: 0.0359
Epoch 2/50
644/644 [==============================] - 22s 35ms/step - loss: 6.5013 - accuracy: 0.0342 - val_loss: 6.7249 - val_accuracy: 0.0361
Epoch 3/50
644/644 [==============================] - 56s 88ms/step - loss: 6.4056 - accuracy: 0.0391 - val_loss: 6.8017 - val_accuracy: 0.0449
Epoch 4/50
644/644 [==============================] - 66s 102ms/step - loss: 6.3175 - accuracy: 0.0436 - val_loss: 6.8384 - val_accuracy: 0.0534
